In [1]:
import os
import pandas as pd
os.chdir(r'C:\Users\신상훈\Desktop\preprocessing\part-5.-머신러닝-모델의-성능-향상을-위한-전처리\5. 머신러닝 모델의 성능 향상을 위한 전처리\데이터')

In [3]:
df = pd.read_csv('australian.csv')

In [8]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,1,2208,1146,2,4,4,1585,0,0,0,1,2,100,1213,0
1,0,2267,7,2,8,4,165,0,0,0,0,2,160,1,0
2,0,2958,175,1,4,4,125,0,0,0,1,2,280,1,0
3,0,2167,115,1,5,3,0,1,1,11,1,2,0,1,1
4,1,2017,817,2,6,4,196,1,1,14,0,2,60,159,1


In [4]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [5]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [6]:
Train_X.shape # 샘플 대비 특징이 약간 더 많은 정도

(517, 14)

In [7]:
# 특징 선택 전 성능 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.5454545454545455


In [9]:
# 유니크한 값의 개수를 바탕으로 연속형과 이진형 변수 구분
continuous_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) > 3]
binary_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) <= 3]

In [10]:
# 연속형 변수에 대해서는 f_classif을, 이진형 변수에 대해서는 chi2를 적용
from sklearn.feature_selection import *

# f_regression(X, Y) => (statistics, p-value)
continous_cols_pvals = f_classif(Train_X[continuous_cols], Train_Y)[1]
binary_cols_pvals = chi2(Train_X[binary_cols], Train_Y)[1]

In [13]:
continous_cols_pvals

array([1.03434439e-01, 2.51201626e-01, 1.60157465e-14, 1.55992088e-06,
       4.16080760e-05, 2.17672994e-21, 1.04003212e-02, 3.41176185e-07])

In [14]:
# 각각을 Series로 변환 (value: pvalue, index: colum name)
cont_pvals = pd.Series(continous_cols_pvals, index = continuous_cols)
binary_pvals = pd.Series(binary_cols_pvals, index = binary_cols)

In [15]:
# cont_pvals과 binary_pvals을 합친 뒤, 오름차순으로 정렬 (앞에 나오는 특징부터 좋은 특징)
pvals = pd.concat([cont_pvals, binary_pvals])
pvals.sort_values(ascending = True, inplace = True)

In [16]:
# 특징 선택 
k = 10

s_Train_X = Train_X[pvals.iloc[:k].index]
s_Test_X = Test_X[pvals.iloc[:k].index]

In [17]:
# 특징 선택 후 성능 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(s_Train_X, Train_Y)
pred_Y = model.predict(s_Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.5467625899280576
